In [1]:
import tweepy
from pyspark.sql import SparkSession
consumer_key = "Ax79DI6gXdMjOCQyQ61v2Khbj"
consumer_secret = "DBSmfUL50FlhpVnePa373lOpBFffJjrGInXbSGG4T7Ze2RRQTl"
access_token = "1418173405988376579-9g3SF5klnWOgd62nUfTfaqrQdzkJRA"
access_token_secret = "q3jO28xbGDlvE6djCNeoSMQbtkosZYT7mBTv0O6wznMpw"
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


In [2]:
spark = SparkSession.builder.appName('logistic').getOrCreate()
df = spark.read.csv("trainingandtestdata/training.1600000.processed.noemoticon.csv")
df = df.withColumnRenamed("_c0", 'polarity')
df = df.withColumnRenamed("_c1", 'tweet_id')
df = df.withColumnRenamed("_c2", 'tweet_date')
df = df.withColumnRenamed("_c3", 'query_string')
df = df.withColumnRenamed("_c4", 'tweet_screen_name')
df = df.withColumnRenamed("_c5", 'text')
df.show(5)

+--------+----------+--------------------+------------+-----------------+--------------------+
|polarity|  tweet_id|          tweet_date|query_string|tweet_screen_name|                text|
+--------+----------+--------------------+------------+-----------------+--------------------+
|       0|1467810369|Mon Apr 06 22:19:...|    NO_QUERY|  _TheSpecialOne_|@switchfoot http:...|
|       0|1467810672|Mon Apr 06 22:19:...|    NO_QUERY|    scotthamilton|is upset that he ...|
|       0|1467810917|Mon Apr 06 22:19:...|    NO_QUERY|         mattycus|@Kenichan I dived...|
|       0|1467811184|Mon Apr 06 22:19:...|    NO_QUERY|          ElleCTF|my whole body fee...|
|       0|1467811193|Mon Apr 06 22:19:...|    NO_QUERY|           Karoli|@nationwideclass ...|
+--------+----------+--------------------+------------+-----------------+--------------------+
only showing top 5 rows



In [3]:
df=df[['polarity','text']]
df.show(5)

+--------+--------------------+
|polarity|                text|
+--------+--------------------+
|       0|@switchfoot http:...|
|       0|is upset that he ...|
|       0|@Kenichan I dived...|
|       0|my whole body fee...|
|       0|@nationwideclass ...|
+--------+--------------------+
only showing top 5 rows



In [4]:
# Follow every follower of the authenticated user
timeline = api.home_timeline()
for tweet in timeline:
    print(f"{tweet.user.name} said {tweet.text}")
    print(tweet.id)
    print(tweet.created_at)
    print(tweet.user.screen_name)
    print(tweet)

Center for Global Development said As the #IDA20 replenishment gets finalized, important ❔'s remain:

How big should the IDA be❔ Does it focus on the… https://t.co/yNBDNK5jmA
1465369220208898058
2021-11-29 17:17:01+00:00
CGDev
Status(_api=<tweepy.api.API object at 0x0000026F0E819EE0>, _json={'created_at': 'Mon Nov 29 17:17:01 +0000 2021', 'id': 1465369220208898058, 'id_str': '1465369220208898058', 'text': "As the #IDA20 replenishment gets finalized, important ❔'s remain:\n\nHow big should the IDA be❔ Does it focus on the… https://t.co/yNBDNK5jmA", 'truncated': True, 'entities': {'hashtags': [{'text': 'IDA20', 'indices': [7, 13]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/yNBDNK5jmA', 'expanded_url': 'https://twitter.com/i/web/status/1465369220208898058', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [116, 139]}]}, 'source': '<a href="https://sproutsocial.com" rel="nofollow">Sprout Social</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_st